In [3]:
import pandas as pd
# 파일 경로 설정
merged_data_path = 'C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/merged_data.csv'
merged_data= pd.read_csv(merged_data_path, encoding='cp949')

### 피처 엔지니어링
1.온도 차이 (Temperature Range)

최고기온과 최저기온의 차이는 날씨의 변동성

2.평균풍속 및 최대풍속의 비율 (Wind Speed Ratio)

평균풍속 대비 최대풍속의 비율은 바람의 강도 변동을 나타내며, 강풍이 자주 발생하는 날씨의 특성을 포착

In [4]:
# 온도 차이 계산
merged_data['temp_range'] = merged_data['최고기온(℃)'] - merged_data['최저기온(℃)']

# 평균풍속 및 최대풍속의 비율 계산
merged_data['wind_speed_ratio'] = merged_data['최대풍속(m/s)'] / merged_data['평균풍속(m/s)']

In [5]:
merged_data

,날짜,평균기온(℃),최저기온(℃),최고기온(℃),\t\t지점번호_x,평균풍속(m/s),최대풍속(m/s),최대풍속풍향(deg),최대순간풍속(m/s),최대순간풍속풍향(deg),평균습도(%rh),최저습도(%rh),일조합(hr),일조율(%),일사합(MJ/m2),강수량(mm),temp_range,wind_speed_ratio
0,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0,8.1,1.739130
1,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0,8.1,1.739130
2,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0,8.1,1.739130
3,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0,8.1,1.739130
4,2000-01-01,5.5,1.8,9.9,108,2.3,4.0,50.0,7.4,50.0,63.5,46.0,2.3,24.0,4.46,0.0,8.1,1.739130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17997,2024-05-04,21.4,14.6,28.5,108,1.9,4.3,250.0,7.7,270.0,44.8,27.0,12.5,89.9,26.61,0.0,13.9,2.263158
17998,2024-05-05,17.8,15.7,20.6,108,2.5,5.6,200.0,10.7,180.0,87.4,44.0,0.0,0.0,2.17,48.3,4.9,2.240000
17999,2024-05-06,13.9,13.4,15.7,108,3.1,4.9,200.0,9.4,200.0,92.9,88.0,0.0,0.0,2.71,12.3,2.3,1.580645
18000,2024-05-07,13.3,12.2,15.0,108,3.0,4.9,70.0,9.5,70.0,77.3,60.0,0.0,0.0,4.75,2.4,2.8,1.633333


날짜' 열을 사용하여 계절성을 나타내는 새로운 이진 피처들을 생성

각 계절은 봄, 여름, 가을, 겨울로 구분하고, 해당 계절에 맞는 행에 1을, 아닌 경우에는 0을 할당

In [6]:
# '날짜' 열을 datetime 형식으로 안전하게 변환
merged_data['날짜'] = pd.to_datetime(merged_data['날짜'], errors='coerce')

In [7]:
# 날짜 열에서 월 정보 추출
if merged_data['날짜'].dtype == 'datetime64[ns]':  # datetime 타입인지 확인
    merged_data['month'] = merged_data['날짜'].dt.month

    # 계절성을 나타내는 새로운 이진 피처 생성
    def assign_season(month):
        if month in [3, 4, 5]:
            return 1, 0, 0, 0  # 봄
        elif month in [6, 7, 8]:
            return 0, 1, 0, 0  # 여름
        elif month in [9, 10, 11]:
            return 0, 0, 1, 0  # 가을
        else:
            return 0, 0, 0, 1  # 겨울

    # apply 함수를 사용하여 한 번에 모든 계절 피처를 DataFrame에 추가
    merged_data[['is_spring', 'is_summer', 'is_fall', 'is_winter']] = merged_data['month'].apply(lambda x: pd.Series(assign_season(x)))

In [8]:
# 결과 출력
print(merged_data.head())

          날짜  평균기온(℃)  최저기온(℃)  최고기온(℃)  \t\t지점번호_x  평균풍속(m/s)  최대풍속(m/s)  \
0 2000-01-01      5.5      1.8      9.9         108        2.3        4.0   
1 2000-01-01      5.5      1.8      9.9         108        2.3        4.0   
2 2000-01-01      5.5      1.8      9.9         108        2.3        4.0   
3 2000-01-01      5.5      1.8      9.9         108        2.3        4.0   
4 2000-01-01      5.5      1.8      9.9         108        2.3        4.0   

   최대풍속풍향(deg)  최대순간풍속(m/s)  최대순간풍속풍향(deg)  ...  일조율(%)  일사합(MJ/m2)  강수량(mm)  \
0         50.0          7.4           50.0  ...    24.0        4.46      0.0   
1         50.0          7.4           50.0  ...    24.0        4.46      0.0   
2         50.0          7.4           50.0  ...    24.0        4.46      0.0   
3         50.0          7.4           50.0  ...    24.0        4.46      0.0   
4         50.0          7.4           50.0  ...    24.0        4.46      0.0   

   temp_range  wind_speed_ratio  month  is_spring  is_su

### 습도의 일일 변동성

In [9]:
# 평균 습도와 최저 습도의 차이를 계산하여 습도 변동성 피처 생성
merged_data['humidity_variance'] = merged_data['평균습도(%rh)'] - merged_data['최저습도(%rh)']

In [10]:
# 전치리 완료된 데이터 프레임을 CSV 파일로 저장
merged_data.to_csv('C:/Users/82106/Desktop/데이터 분석 프로젝트 2/날씨 빅데이터 콘테스트/데이터/merged_data.csv', index=False, encoding='cp949')